In [99]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
# from main import get_normalized_probability, cal_most_prob_scene, reverse_relation
import json
import pulp
import numpy as np
from LPConstraints import get_clevr_block_world_constraint

In [200]:
with open('../results/clevr_block_val/constraint_1_2_3_4/rel_scenes.json', 'r') as f:
    scenes = json.load(f)['scenes']

with open('../data/clevr_block/clevr_attr_map.json', 'r') as f:
    attr_map = json.load(f)

In [201]:
scene = scenes[0]

In [202]:
def get_attribute_variables(objects, attributes_map):
    variables = []
    probabilities = {}
    num_objects = len(objects)

    for name, values in attributes_map.items():
        for v, _ in enumerate(values):
            for i in range(num_objects):
                variable_name = f'{name}_{i}_{v}'
                variables.append(variable_name)
                probabilities[variable_name] = objects[i][name][v]
    return variables, probabilities

def get_relationship_variables(scene, relationships):
    variables = []
    probabilities = {}

    for rel in relationships:
        for target, sources in enumerate(scene['relationships'][rel]):
            for source, probability in sources:
                # TODO: change the prediction for below and above (currently flipped)
                variable_name = f'{rel}_{source}_{target}'
                variables.append(variable_name)
                probabilities[variable_name] = probability
    return variables, probabilities

def get_LP_problem(scene, attr_map, eps=1e-50):
    attr_variables, attr_probabilities = get_attribute_variables(scene['objects'], attr_map['attributes'])
    rel_variables, rel_probabilities = get_relationship_variables(scene, attr_map['relations'])


    attr_variables = pulp.LpVariable.dict('attr', attr_variables, 0, 1, pulp.LpBinary)
    rel_variables = pulp.LpVariable.dict('rel', rel_variables, 0, 1, pulp.LpBinary)

    attr_objective = [attr_variables[i] * np.log(max(attr_probabilities[i], eps)) for i in attr_variables.keys()]
    rel_objective = [rel_variables[i] * np.log(max(rel_probabilities[i], eps)) + (1 - rel_variables[i]) * np.log((max(1 - rel_probabilities[i], eps))) for i in rel_variables.keys()]

    prob = pulp.LpProblem("sceneGraphProblem", pulp.LpMaximize)
    prob += pulp.lpSum(attr_objective + rel_objective)
    return prob, attr_variables, rel_variables

def get_predicted_scene_for_block_world(variables):
    predicted_scene = {
        'objects': [{} for _ in range(len(scene['objects']))],
        'relationships': { rel: [[] for _ in range(len(scene['objects']))] for rel in attr_map['relations']}
    }

    for v in variables:
        if v.varValue:
            tokens = v.name.split('_')
            if tokens[0] == 'attr':
                predicted_scene['objects'][int(tokens[2])][tokens[1]] = attr_map['attributes'][tokens[1]][int(tokens[3])]
            elif tokens[0] == 'rel':
                predicted_scene['relationships'][tokens[1]][int(tokens[3])].append(int(tokens[2]))
    return predicted_scene

In [205]:
from tqdm import tqdm
M = 100
predicted_scenes = []

for scene in tqdm(scenes):
    prob, attr_variables, rel_variables = get_LP_problem(scene, attr_map)
    get_clevr_block_world_constraint(prob, scene, rel_variables, attr_variables, attr_map, include_small_obj=True, include_large_cube=True)
    prob.solve()
    predicted_scenes.append(get_predicted_scene_for_block_world(prob.variables()))

100%|██████████| 1000/1000 [00:53<00:00, 18.64it/s]


In [207]:
with open('../results/clevr_block_val/constraint_1_2_3_4/rel_scenes_consistent_1_2.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)



In [173]:
attr_variables

{'shape_0_0': attr_shape_0_0,
 'shape_1_0': attr_shape_1_0,
 'shape_2_0': attr_shape_2_0,
 'shape_0_1': attr_shape_0_1,
 'shape_1_1': attr_shape_1_1,
 'shape_2_1': attr_shape_2_1,
 'shape_0_2': attr_shape_0_2,
 'shape_1_2': attr_shape_1_2,
 'shape_2_2': attr_shape_2_2,
 'size_0_0': attr_size_0_0,
 'size_1_0': attr_size_1_0,
 'size_2_0': attr_size_2_0,
 'size_0_1': attr_size_0_1,
 'size_1_1': attr_size_1_1,
 'size_2_1': attr_size_2_1,
 'material_0_0': attr_material_0_0,
 'material_1_0': attr_material_1_0,
 'material_2_0': attr_material_2_0,
 'material_0_1': attr_material_0_1,
 'material_1_1': attr_material_1_1,
 'material_2_1': attr_material_2_1,
 'color_0_0': attr_color_0_0,
 'color_1_0': attr_color_1_0,
 'color_2_0': attr_color_2_0,
 'color_0_1': attr_color_0_1,
 'color_1_1': attr_color_1_1,
 'color_2_1': attr_color_2_1,
 'color_0_2': attr_color_0_2,
 'color_1_2': attr_color_1_2,
 'color_2_2': attr_color_2_2,
 'color_0_3': attr_color_0_3,
 'color_1_3': attr_color_1_3,
 'color_2_3': at

In [203]:
print(len(prob.constraints))

174


In [169]:
rel_variables[f'left_{3}_{5}'].varValue

1.0

In [154]:
scene_file = '../data/clevr_block/constraint_1_2_3_4/scenes.json'

with open(scene_file) as f:
    gt_scenes = json.load(f)

In [155]:
for scene in gt_scenes['scenes']:
    scene['relationships']['above'], scene['relationships']['below'] = scene['relationships']['below'], scene['relationships']['above']

In [156]:
with open(scene_file, 'w') as f:
    json.dump(gt_scenes, f)